In [ ]:
def sentences_to_conll(sentences, model, arc_dict):
    action_dict = {v:k for k,v in arc_dict.items()}
    all_sentences_listed = []
    for sentence in sentences:
        sentence_parse = [[i+1,word,'_','_','_','_','_','_','_','_'] for i, word in enumerate(sentence)]
        w2v_matrix = create_sentence_embeddings([sentence])
        stack = []
        buff = list(range(1,len(sentence)+1))
        sentence_parse, = single_sentence_parse(stack, buff, sentence_parse, action_dict, w2v_matrix, model)
        all_sentences_listed.append(sentence_parse)
    conll = convert_to_conll(all_sentences_listed)
    return conll
        
        
def single_sentence_parse(stack, buff, sentence_parse, action_dict, w2v_matrix, model):
    #If there are 2 or more words in the stack, decide which action to perform and perform it
    if len(stack) > 1:
        s1 = int(stack[-2])
        s2 = int(stack[-1])
        #checks whether buffer contains words
        if len(buff) > 0:
            b1 = int(buff[0])
            action = model_action_decision(w2v_matrix,s1,s2,b1,model,False)
        else:
            b1 = 0
            action = model_action_decision(w2v_matrix,s1,s2,b1,model,True)
        
        if action == 0:
            # perform a shift
            stack, buff = Shift(stack, buff)
        elif action%2 == 1:
            # left-arc. All left tags are odd in the dictionary
            stack, sentence_parse = L_arc(stack,s1,s2, sentence_parse, action_dict, action)
        else:
            # right-arc. All right tags are even in the dictionary
            stack, sentence_parse = R_arc(stack,s1,s2, sentence_parse, action_dict, action)
        return single_sentence_parse(stack, buff, sentence_parse, action_dict, w2v_matrix, model)
    
    #base case (R_arc): if only one word is left, perform the last right arc with root.
    if len(stack) == 1 and len(buff) == 0:
        sentence_parse[stack[0]-1][6] = 0
        sentence_parse[stack[0]-1][7] = 'root'
        return sentence_parse    

    #If there is not enough material in the stack, shift:
    if len(stack) == 0 :
        #print('chose S - small stack')
        stack, buff = Shift(stack, buff)       
        return single_sentence_parse(stack, buff, sentence_parse, action_dict, w2v_matrix, model)
    if len(stack) == 1:
        stack, buff = Shift(stack, buff)
        return single_sentence_parse(stack, buff, sentence_parse, action_dict, w2v_matrix, model)
    
#TODO1: create func.
def model_action_decision(w2v_matrix,s1,s2,b1,model,emptybuffer):
    #if emptybuffer is true, exclude option 0 (shift).
    if emptybuffer == False:
        session.run(...)
    else:
        
    
def L_arc(stack,s1,s2, sentence_parse, action_dict, action):
    #removes second to last item from stack, sends info to sentence_parse

    action_type = action_dict[action]
    
    #update head and relation for s1
    sentence_parse[s1-1][6] = s2
    sentence_parse[s1-1][7] = action_type[2:]
    
    del stack[-2]
    return stack, sentence_parse


def R_arc(stack,s1,s2, sentence_parse, action_dict, action):
    #removes last item from the stack, sends info to sentence_parse
    
    action_type = action_dict[action]

    #update head and relation for s2
    sentence_parse[s2-1][6] = s1
    sentence_parse[s2-1][7] = action_type[2:]
    
    del stack[-1]
    return stack, sentence_parse


def Shift(stack, buff):
    #moves an item from the buff to the stack
    #input: stack, buff
    #output: new stack and buff
    stack.append(buff[0])
    del buff[0]
    return stack, buff

#TODO2: func2
def convert_to_conll():